# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [1]:
# import library and data set
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [2]:
data = pd.read_csv('data/hr_test.csv')

In [3]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [4]:
data = remove_random(data, .25)

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.41,4.0,132.0,5.0,1.0,0.0,0.0,support,medium
1,NaN,0.87,6.0,232.0,5.0,0.0,0.0,0.0,management,high
2,NaN,0.91,7.0,287.0,4.0,0.0,1.0,0.0,sales,low
3,NaN,0.98,4.0,180.0,3.0,0.0,0.0,0.0,support,medium
4,NaN,0.78,5.0,131.0,8.0,0.0,0.0,0.0,support,low


In [6]:
len(data)

14999

In [7]:
reload(impyte)
imputer = impyte.Imputer() # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()

### Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

#### NanChecker.is_nan( )
The NaN detection feature that also enables NaN-value ingestion as parameter values.

In [8]:
# [True, False, False]
print nan_checker.is_nan(["", 'None', 'NaN'])

# [True, True, True]
print nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


#### Pattern.row_nan_pattern( )
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [9]:
# (1, 1, 'NaN', 'NaN')
print pattern_log.row_nan_pattern(['yes', 'no', '', None])

(1, 1, 'NaN', 'NaN')


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [10]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

100000 loops, best of 3: 7.24 µs per loop
The slowest run took 4.12 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.84 µs per loop


In [11]:
# Showcase nan_check
%timeit pattern_log.row_nan_pattern(['yes', 'no', '', None])

100000 loops, best of 3: 8.21 µs per loop


In [12]:
%timeit pattern_log.print_pattern(data)

1 loop, best of 3: 745 ms per loop


In [13]:
%timeit pattern_log.compute_pattern(data)

1 loop, best of 3: 1.82 s per loop


#### Pattern.compute_pattern( )
Feature that computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `['table']` selector to output.

In [14]:
pattern_log.compute_pattern(data)['table']

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374


### Load data into imputer

In [15]:
reload(impyte)
imputer.load_data(data)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.41,4.0,132.0,5.0,1.0,0.0,0.0,support,medium
1,NaN,0.87,6.0,232.0,5.0,0.0,0.0,0.0,management,high
2,NaN,0.91,7.0,287.0,4.0,0.0,1.0,0.0,sales,low
3,NaN,0.98,4.0,180.0,3.0,0.0,0.0,0.0,support,medium
4,NaN,0.78,5.0,131.0,8.0,0.0,0.0,0.0,support,low
5,NaN,0.58,3.0,203.0,2.0,0.0,0.0,0.0,RandD,medium
6,NaN,0.99,2.0,258.0,3.0,1.0,0.0,0.0,sales,medium
7,NaN,0.57,4.0,207.0,5.0,0.0,0.0,0.0,sales,low
8,NaN,0.54,4.0,180.0,5.0,0.0,0.0,0.0,sales,high
9,NaN,0.80,6.0,256.0,4.0,0.0,1.0,0.0,IT,low


In [16]:
%timeit imputer.pattern()

1 loop, best of 3: 1.79 s per loop


In [17]:
%timeit imputer.get_pattern(4)

The slowest run took 9.49 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 756 µs per loop


In [36]:
reload(impyte)
imputer = impyte.Imputer(data) # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()
_ = imputer.pattern()
_ = imputer.drop_pattern(4)

In [37]:
#imputer.pattern()

In [38]:
_ = imputer.load_data(data)

In [39]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374


In [40]:
_ = imputer.drop_pattern(4)

In [41]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
9,1,1,1,1,1,1,1,1,1,1,11259
7,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
8,1,1,1,1,1,1,1,1,1,NaN,374
6,1,1,1,1,1,1,1,NaN,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
4,1,1,1,1,1,NaN,1,1,1,1,374
5,1,1,1,1,1,1,NaN,1,1,1,374


In [42]:
_ = imputer.drop_pattern(7)

In [43]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
8,1,1,1,1,1,1,1,1,1,1,11259
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
7,1,1,1,1,1,1,1,1,1,NaN,374
6,1,1,1,1,1,1,1,NaN,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
4,1,1,1,1,1,NaN,1,1,1,1,374
5,1,1,1,1,1,1,NaN,1,1,1,374


In [44]:
imputer.impute()

(['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 ['sales', 'salary'])

In [45]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.52,0.94,3.0,263.0,3.0,0.0,0.0,0.0,product_mng,medium
14995,0.43,0.44,5.0,213.0,3.0,0.0,0.0,0.0,sales,low
14996,0.45,0.49,2.0,134.0,3.0,0.0,1.0,0.0,RandD,medium
14997,0.10,0.90,7.0,281.0,4.0,0.0,1.0,0.0,sales,low
14998,0.54,0.67,2.0,129.0,3.0,1.0,0.0,0.0,sales,low
